# 💬 Claimify reproduction study

In [1]:
# autoreload imports
%load_ext autoreload
%autoreload 2

In [2]:
from utils.chat import load_md_prompts

PROMPT_TEMPLATES = load_md_prompts("claimify")

## 1. Input Question and Answer

In [13]:
%pip install charboundary[numpy,onnx]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 8.3 MB/s  0:00:0010.4 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 12.0 MB/s  0:00:01 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 55.7 MB/s  0:00:00 52.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [charboundary]m━━━━ 8/9 [charboundary]
Note: you may need to restart the kernel to use updated packages.


In [27]:
# basic chatprompttemplate llm chat with gpt-5 using 'selection' prompt template
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

import ssl
import certifi

chat = ChatOpenAI(model="gpt-5")
excerpt = """"""
question = """"""
sentence = """"""
chat.invoke(PROMPT_TEMPLATES["selection"].format_messages(excerpt=excerpt, question=question, sentence=sentence))

AIMessage(content='Sentence: \n<>\n\n4-step stream of consciousness thought process (1. reflect on criteria at a high-level -> 2. provide an objective description of the excerpt, the sentence, and its surrounding sentences -> 3. consider all possible perspectives on whether the sentence explicitly or implicitly contains a specific and verifiable proposition, or if it just contains an introduction for the following \nsentence(s), a conclusion for the preceding sentence(s), broad or generic statements, opinions, interpretations, speculations, statements about a lack of information, etc. -> 4. only if it contains a specific and verifiable proposition: reflect on whether any changes are needed to ensure that the entire sentence only contains verifiable information):\n1) A sentence must assert at least one checkable claim to qualify; absence of content cannot contain a proposition. 2) The provided "Sentence" field is empty, and no surrounding context is given. 3) An empty sentence cannot ex

In [ ]:
from utils.claimify import SelectionResponse

In [8]:
chat = ChatOllama(model="qwen3:14b")
output = chat.invoke(PROMPT_TEMPLATES["selection"].format_messages(excerpt="todo", question="todo", sentence="todo"))

In [12]:
output.content

'<think>\nOkay, let\'s tackle this problem step by step. The user wants me to determine if a given sentence from a response contains a specific and verifiable proposition based on the provided rules.\n\nFirst, I need to recall the criteria. A specific and verifiable proposition is something that can be checked for truth, like a fact. The rules mention that if the sentence is about a lack of information, it doesn\'t count. Also, it doesn\'t matter if the proposition is true or false, just whether it\'s specific and verifiable. Ambiguous terms are okay as long as the fact-checker can resolve them.\n\nNow, looking at the example given in the problem statement. The user provided a scenario where the sentence is "John" following the question "Who is the CEO of Company X?" and the preceding sentence. The answer here is that "John" contains a specific proposition because it directly answers the question with a name, which can be verified.\n\nAnother example is when the preceding sentence ment

In [24]:
from typing import List
import nltk

def split_into_sentences(text: str) -> List[str]:
    """
    Splits a block of text into sentences, handling paragraphs and lists.
    This replicates the methodology from Appendix C.1 of the Claimify paper.
    
    Args:
        text: The input text to split
        
    Returns:
        A list of sentence strings
    """
    
    sentences = []
    # First, split by newlines to handle paragraphs and list items
    paragraphs = text.split('\n')
    for para in paragraphs:
        if para.strip():  # Avoid empty paragraphs
            # Then, use NLTK's sentence tokenizer on each paragraph
            sentences.extend(nltk.sent_tokenize(para))
    return sentences

split_into_sentences("""
List The Wall Street Journal předtím uvedl, že Evropané na schůzi představili vlastní plán jednání o ukončení války. Podle dokumentu před přiznáním jakýchkoli ústupků Rusku je nutné vyhlásit příměří. Eventuální územní ústupky musí být reciproční, musí je tedy udělat Ukrajina i Rusko. Plán také žádá silné bezpečnostní záruky pro Ukrajinu včetně členství v NATO. Trump v minulosti dával najevo, že takový postup si nepřeje.

Bílý dům zvažuje pozvání Zelenského na Aljašku
V pátek 15. srpna se Trump sejde na Aljašce se svým ruským protějškem Vladimirem Putinem. Půjde o dvoustrannou schůzku, Bílý dům však podle informací stanice NBC News stále zvažuje, že na Aljašku pozve také ukrajinského prezidenta Volodymyra Zelenského.

NBC News se odvolává na vysoce postaveného amerického činitele a tři osoby, které byly o interních diskusích v Bílém domě informovány.

„Diskutuje se o tom,“ citovala stanice jednoho z lidí seznámeného s obsahem jednání. Vysoce postavený americký činitel a lidé obeznámení s rozhovory zároveň uvedli, že návštěva Zelenského zatím nebyla s konečnou platností dohodnuta a není jasné, zda se Zelenskyj nakonec jednání na Aljašce zúčastní.

Lipavský o schůzce Trumpa s Putinem: Hranice Ukrajiny neposune vydírání

Vysoce postavený představitel Trumpovy administrativy k tomu řekl, že je to „rozhodně“ možné. „Všichni silně doufají, že k tomu dojde,“ dodal.

Trump podle něj zůstává otevřen možnosti trojstranného summitu s oběma lídry. Momentálně se ale Bílý dům „soustředí na přípravu bilaterální schůzky, o kterou požádal prezident Putin“, řekl činitel.

Ukrajinský prezident Volodymyr Zelenskyj v sobotu varoval, že jakékoliv řešení rusko-ukrajinského konfliktu učiněné bez účasti Ukrajiny bude řešením proti míru. Dodal, že Ukrajinci nevydají svou zemi okupantům.

Šéf Kremlu tento týden prohlásil, že není proti setkání se Zelenským, ale pro takovou schůzku podle něj ještě nejsou podmínky. Zelenskyj opakovaně vyzval k osobnímu jednání mezi ním a Putinem s cílem ukončit ruskou invazi na Ukrajinu.

Ukrajina se brání rozsáhlé ruské vojenské agresi od února 2022. Zatím není jasné, jestli je Moskva připravena vzdát se jakéhokoliv území, které nyní okupuje. Kyjev dosud podle agentury Bloomberg dával najevo, že ruskou okupaci a anexi svého území nepřijme.

Podle agentury Reuters sice Ukrajina signalizovala flexibilitu při hledání cest k ukončení války, ale přijmout ztrátu zhruba pětiny území by bylo velmi bolestivé a politicky složité pro Zelenského a jeho vládu.
)
Zdroj: https://www.idnes.cz/zpravy/zahranicni/zelensky-trump-putin-aljaska-schuzka-usa-rusko-ukrajina-valka.A250810_071424_zahranicni_svm""")

['List The Wall Street Journal předtím uvedl, že Evropané na schůzi představili vlastní plán jednání o ukončení války.',
 'Podle dokumentu před přiznáním jakýchkoli ústupků Rusku je nutné vyhlásit příměří.',
 'Eventuální územní ústupky musí být reciproční, musí je tedy udělat Ukrajina i Rusko.',
 'Plán také žádá silné bezpečnostní záruky pro Ukrajinu včetně členství v NATO.',
 'Trump v minulosti dával najevo, že takový postup si nepřeje.',
 'Bílý dům zvažuje pozvání Zelenského na Aljašku',
 'V pátek 15. srpna se Trump sejde na Aljašce se svým ruským protějškem Vladimirem Putinem.',
 'Půjde o dvoustrannou schůzku, Bílý dům však podle informací stanice NBC News stále zvažuje, že na Aljašku pozve také ukrajinského prezidenta Volodymyra Zelenského.',
 'NBC News se odvolává na vysoce postaveného amerického činitele a tři osoby, které byly o interních diskusích v Bílém domě informovány.',
 '„Diskutuje se o tom,“ citovala stanice jednoho z lidí seznámeného s obsahem jednání.',
 'Vysoce postav

In [11]:
from utils.claimify import ClaimifyPipeline
from utils.chat import chat_factory

pipeline = ClaimifyPipeline(chat_factory("gpt-5-mini"))

2025-08-10 12:38:06,040 - claimify.pipeline - INFO - Structured outputs enabled for improved reliability


In [12]:
article = """Mír bez Ukrajiny nepůjde, zdůrazňuje Evropa. Zelenskyj by mohl na Aljašku
Cestu k míru na Ukrajině nelze určit bez Ukrajiny, uvedli ve společném prohlášení lídři Británie, Finska, Francie, Itálie, Německa a Polska a předsedkyně Evropské komise Ursula von der Leyenová. Bílý dům zvažuje, že pozve ukrajinského prezidenta Volodymyra Zelenského na Aljašku, kde se má americký prezident Donald Trump 15. srpna sejít se svým ruským protějškem Vladimirem Putinem."""

In [8]:
print("cau")

cau


In [13]:
result = pipeline.run(article)
result

2025-08-10 12:38:43,571 - claimify.pipeline - INFO - Processing 4 sentences
2025-08-10 12:38:43,571 - claimify.pipeline - INFO - Processing sentence 1/4: Mír bez Ukrajiny nepůjde, zdůrazňuje Evropa....
2025-08-10 12:38:52,805 - claimify.pipeline - INFO - SELECTION: Sentence verifiable
2025-08-10 12:39:07,959 - claimify.pipeline - INFO - DISAMBIGUATION: Sentence resolved
2025-08-10 12:39:27,134 - claimify.pipeline - INFO - DECOMPOSITION: Extracted 3 claims
2025-08-10 12:39:27,134 - claimify.pipeline - INFO - Processing sentence 2/4: Zelenskyj by mohl na Aljašku...
2025-08-10 12:39:39,491 - claimify.pipeline - INFO - SELECTION: Sentence verifiable
2025-08-10 12:39:53,742 - claimify.pipeline - INFO - DISAMBIGUATION: Sentence resolved
2025-08-10 12:40:09,230 - claimify.pipeline - INFO - DECOMPOSITION: Extracted 2 claims
2025-08-10 12:40:09,230 - claimify.pipeline - INFO - Processing sentence 3/4: Cestu k míru na Ukrajině nelze určit bez Ukrajiny, uvedli ve společném prohlášení lídři Britán

['The leaders of Britain, Finland, France, Italy, Germany, and Poland and European Commission President Ursula von der Leyen issued a joint statement - true or false?',
 'In that joint statement, those leaders and Ursula von der Leyen said that the path to peace in Ukraine cannot be determined without Ukraine - true or false?',
 'Ursula von der Leyen is the President of the European Commission - true or false?',
 'Volodymyr Zelenskyj is the President of Ukraine. - true or false?',
 'Volodymyr Zelenskyj might travel to Alaska [based on a report stating that the White House is considering inviting him to Alaska where US President Donald Trump is scheduled to meet Russian President Vladimir Putin on August 15] - true or false?',
 'The leaders of Britain [as part of a joint statement issued by the leaders of Britain, Finland, France, Italy, Germany, Poland, and European Commission President Ursula von der Leyen] said that the path to peace in Ukraine cannot be determined without Ukraine - 

In [53]:
from utils.chat import get_prompt
prompt = get_prompt("claimify", "disambiguation", {"question": question, "excerpt": excerpt, "sentence": sentence})
# type of prompt
prompt

[SystemMessage(content='You are an assistant to a fact-checker. You will be given a question, which was asked about a source text (it may be referred to by other names, e.g., a disa\ndataset). You will also be given an excerpt from a response to the question. If it contains "[...]", this means that you are NOT seeing all sentences in the response. You will also be given a particular sentence from the response. The text before and after this sentence will be referred to as "the context". Your task is to "decontextualize" the sentence, which means:\n1. determine whether it\'s possible to resolve partial names and undefined acronyms/abbreviations in the sentence using the question and the context; if it is possible, you will make the necessary changes to the sentence\n2. determine whether the sentence in isolation contains linguistic ambiguity that has a clear resolution using the question and the context; if it does, you will make the necessary changes to the sentence\n\nNote the followi

In [49]:
get_prompt("claimify", "selection", {"question": question, "excerpt": excerpt, "sentence": sentence})

[SystemMessage(content='You are an assistant to a fact-checker. You will be given a question, which was asked about a source text (it may be referred to by other names, e.g., a \ndataset). You will also be given an excerpt from a response to the question. If it contains "[...]", this means that you are NOT seeing all sentences in the response. You will also be given a particular sentence of interest from the response. Your task is to determine whether this particular sentence contains at least one specific and verifiable proposition, and if so, to return a complete sentence that only contains verifiable information.   \n\nNote the following rules:\n- If the sentence is about a lack of information, e.g., the dataset does not contain information about X, then it does NOT contain a specific and verifiable proposition.\n- It does NOT matter whether the proposition is true or false.\n- It does NOT matter whether the proposition is relevant to the question.\n- It does NOT matter whether the 